In [139]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
print(spark)

In [140]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col 
book_metadata= spark.read.format('csv').option('header','true').load('book_metadata.csv')
book_genre= spark.read.format('csv').option('header','true').load('book_genres.csv')
book_descriptions= spark.read.format('csv').option('header','true').load('book_descriptions.csv')


book_metadata.createOrReplaceTempView('book_metadata_view')
book_genre.createOrReplaceTempView('book_genre_view')
book_descriptions.createOrReplaceTempView('book_descriptions_view')

print(book_metadata)
print(book_genre)
print(book_descriptions)

DataFrame[_c0: string, Title: string, Author: string, Score: string, Ratings: string, Published: string]
DataFrame[_c0: string, Title: string, Genre: string]
DataFrame[_c0: string, Title: string, Description: string]


In [141]:
result1 = book_metadata.join(book_descriptions, book_metadata['Title'] == book_descriptions['Title'], "inner").drop(book_descriptions['Title'],book_descriptions['_c0'])
print(type(result1),type(book_metadata))
result1.show()

<class 'pyspark.sql.dataframe.DataFrame'> <class 'pyspark.sql.dataframe.DataFrame'>
+----+--------------------+--------------------+-----+-------+---------+--------------------+
| _c0|               Title|              Author|Score|Ratings|Published|         Description|
+----+--------------------+--------------------+-----+-------+---------+--------------------+
|1542|Midnight Sun (The...|     Stephenie Meyer| 3.73| 223895|     2020|When Edward Culle...|
| 784|   Out of the Corner|      Jennifer  Grey| 3.81|   8843|     2022|A deeply candid a...|
|1662|               Fangs|      Sarah Andersen| 4.26|  36311|     2020|A love story betw...|
|1141|    You Can Trust Me|    Kiersten Modglin| 4.09|   2232|     2023|For two happy cou...|
|2550|Authentically Izz...|    Pepper D. Basham| 3.89|   2271|     2022|“Dear Izzy—I feel...|
|2811|            Het plan|         Nando Boers| 4.16|     67|     2023|Het plan geeft de...|
|2104|                Kala|         Colin Walsh| 4.17|    798|     202

In [142]:
result=result1.join(book_genre,result1['Title']==book_genre['Title'],"inner").drop(book_genre['Title'],book_genre['_c0'],result1['_c0'])
print(result)
result.show()

DataFrame[Title: string, Author: string, Score: string, Ratings: string, Published: string, Description: string, Genre: string]
+--------------------+--------------------+-----+-------+---------+--------------------+--------------------+
|               Title|              Author|Score|Ratings|Published|         Description|               Genre|
+--------------------+--------------------+-----+-------+---------+--------------------+--------------------+
|Midnight Sun (The...|     Stephenie Meyer| 3.73| 223895|     2020|When Edward Culle...|  Fantasy,Paranormal|
|   Out of the Corner|      Jennifer  Grey| 3.81|   8843|     2022|A deeply candid a...|           Biography|
|               Fangs|      Sarah Andersen| 4.26|  36311|     2020|A love story betw...|      Graphic Novels|
|    You Can Trust Me|    Kiersten Modglin| 4.09|   2232|     2023|For two happy cou...|   Suspense,Thriller|
|Authentically Izz...|    Pepper D. Basham| 3.89|   2271|     2022|“Dear Izzy—I feel...|           Chr

In [143]:
output_path = "results.csv"


# Write the DataFrame to a text file
# Assuming result is your DataFrame
result.coalesce(1).write.csv(output_path, header=True, mode="overwrite")

In [144]:
import csv

# Input CSV file path
csv_file_path = "results.csv/part-00000-74e6199a-f186-4b5c-b046-e10ecbd4aa32-c000.csv"

# Output text file path
text_file_path = "C:/Users/s556452/Downloads/mapreduce-mvn-main/mapreduce-mvn-main/data/input/output.txt"

from pyspark.sql import SparkSession
import pandas as pd

# Create a Spark session
spark = SparkSession.builder.appName("CSVtoText").getOrCreate()

# Replace 'your_input.csv' with the path to your CSV file
csv_file_path = "results.csv/part-00000-4b173e62-f130-4b3a-bd00-40318816976a-c000.csv"

# Read CSV file into a Pandas DataFrame
pandas_df = pd.read_csv(csv_file_path)

# Convert Pandas DataFrame to PySpark DataFrame
spark_df = spark.createDataFrame(pandas_df)

# Replace 'your_output.txt' with the desired path for the text file
text_file_path = "C:/Users/s556452/Downloads/mapreduce-mvn-main/mapreduce-mvn-main/data/input/output.txt"

# Write the PySpark DataFrame to a text file
spark_df.write.text(text_file_path)

# Stop the Spark session
spark.stop()

C:\Users\s556452\big-data-venv\lib\site-packages\pyspark\sql\pandas\conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


AnalysisException: [UNSUPPORTED_DATA_TYPE_FOR_DATASOURCE] The Text datasource doesn't support the column `Score` of the type "DOUBLE".

In [151]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Create a Spark session
spark = SparkSession.builder.appName("ConvertToText").getOrCreate()

# Sample DataFrame
df= spark.read.format('csv').option('header','true').load('results.csv/part-00000-4b173e62-f130-4b3a-bd00-40318816976a-c000.csv')



# Convert Score, Ratings, and Published to STRING
df = df.withColumn("Score", F.col("Score").cast("string"))
df = df.withColumn("Ratings", F.col("Ratings").cast("string"))
df = df.withColumn("Published", F.col("Published").cast("string"))



df = df.withColumn("all_columns", F.concat_ws("\t", *df.columns))

print(df.columns)

# # Specify the path to the output text file
output_path = "C:/Users/s556452/Downloads/mapreduce-mvn-main/mapreduce-mvn-main/data/input/output.txt"

# Write DataFrame to a text file
df.select("all_columns").write.text(output_path)

# Stop the Spark session
spark.stop()


['Title', 'Author', 'Score', 'Ratings', 'Published', 'Description', 'Genre', 'all_columns']
